In [11]:
import numpy as np
import pandas as pd
import numpy as np
import RQP_test_data_creator
from scipy.io import arff


def sample_interval_from_data(data_train):
    n = data_train.shape[0]
    num_features = data_train.shape[1]
    x1 = data_train[np.random.choice(n)]
    x2 = data_train[np.random.choice(n)]
    feature_intervals = []
    for feature in range(num_features):
        lower_bound = np.min([x1[feature], x2[feature]])
        upper_bound = np.max([x1[feature], x2[feature]])
        feature_intervals.append((lower_bound, upper_bound))
    return feature_intervals


file_path_train = "RQPdata/boston.arff_RQPtrain.arff"
file_path_test = "RQPdata/boston.arff_RQPtest.arff"
data_train_raw = pd.DataFrame(arff.loadarff(file_path_train)[0])
data_test_raw = pd.DataFrame(arff.loadarff(file_path_test)[0])
X_train_raw = data_train_raw.iloc[:, :-1]

aug_sample_numbers = [200, 500, 1000, 2000]
num_features = X_train_raw.shape[1]
print("# Features = ", num_features)
data_train_aug = []
for i in range(aug_sample_numbers[1]):
    feature_intervals = sample_interval_from_data(X_train_raw.values)
    y_hat_min = np.inf
    y_hat_max = -np.inf
    for index, data_point in data_train_raw.iterrows():
        in_interval = True
        for feature in range(num_features):
            feature_interval = feature_intervals[feature]
            if feature_interval[0] > data_point[feature] or feature_interval[1] < data_point[feature]:
                in_interval = False
        if in_interval:
            y_hat_min = np.min([y_hat_min, data_point[-1]])
            y_hat_max = np.max([y_hat_max, data_point[-1]])
    #if not y_hat_min == np.inf and not y_hat_max == -np.inf:
    aug_point = []
    for interval in feature_intervals:
        aug_point.append(interval[0])
        aug_point.append(interval[1])
    aug_point.append(y_hat_min)
    aug_point.append(y_hat_max)
    data_train_aug.append(aug_point)
data_train_aug = pd.DataFrame(data_train_aug)

print(data_train_aug.describe())

X_test = data_test_raw.iloc[:, :-2]
Y_test_min = data_test_raw.iloc[:, -2:]
Y_test_max = data_test_raw.iloc[:, :-1]

# Features =  13
               0           1           2           3           4           5   \
count  500.000000  500.000000  500.000000  500.000000  500.000000  500.000000   
mean     0.008522    0.073888    0.020640    0.195830    0.260426    0.539981   
std      0.022799    0.124284    0.083151    0.272624    0.192501    0.218922   
min      0.000000    0.000160    0.000000    0.000000    0.000000    0.063050   
25%      0.000480    0.003652    0.000000    0.000000    0.119593    0.338343   
50%      0.001098    0.026668    0.000000    0.000000    0.206012    0.646628   
75%      0.003749    0.103316    0.000000    0.330000    0.346041    0.646628   
max      0.188890    1.000000    0.850000    0.950000    0.785557    1.000000   

               6         7           8           9      ...              18  \
count  500.000000  500.0000  500.000000  500.000000     ...      500.000000   
mean     0.014000    0.1420    0.229120    0.502409     ...        0.260664   
std      0.11760

In [19]:
from sklearn.ensemble import RandomForestRegressor

X_train_aug = data_train_aug.iloc[:, :-2]
Y_min_aug = np.ravel(data_train_aug.iloc[:, -2:-1])
Y_max_aug = np.ravel(data_train_aug.iloc[:, -1:])

min_classifier = RandomForestRegressor()
min_classifier = min_classifier.fit(X_train_aug, Y_min_aug)

max_classifier = RandomForestRegressor()
max_classifier = max_classifier.fit(X_train_aug, Y_max_aug)

X_test = data_test_raw.iloc[:, :-2]
Y_test_min = data_test_raw.iloc[:, -2:-1]
Y_test_max = data_test_raw.iloc[:, -1:]

print("Min error:", min_classifier.score(X_test, Y_test_min))
print("Max error:", max_classifier.score(X_test, Y_test_max))

Min error: 0.49706278068711424
Max error: 0.8105183591813654
